In [2]:
from urllib.request import urlopen
import warnings
import os
import json
import pandas as pd

# URL = 'http://www.oreilly.com/pub/sc/osconfeed'
# JSON = '/data/osconfeed.json'

# def load():
#     if not os.path.exists(JSON):
#         msg = 'downloading {} to {}'.format(URL, JSON)
#         warnings.warn(msg)
#         with urlopen(URL) as remote, open(JSON, 'wb') as local:
#             local.write(remote.read())
            
#     with open(JSON) as fp:
#         return json.load(fp)
    
# feed = load()

feed = pd.read_json('https://raw.githubusercontent.com/fluentpython/example-code/master/19-dyn-attr-prop/oscon/data/osconfeed.json')

In [3]:
sorted(feed['Schedule'].keys())

['conferences', 'events', 'speakers', 'venues']

In [4]:
for key, value in sorted(feed['Schedule'].items()):
    print('{:3} {}'.format(len(value), key))

  1 conferences
484 events
357 speakers
 53 venues


In [5]:
feed['Schedule']['events'][40]['name']

'There *Will* Be Bugs'

In [16]:
from collections import abc
import keyword

class FrozenJSON:
    # read only facade for navigating json-like object
    # using attribute notation
    
    def __new__(cls, arg):
        if isinstance(arg, abc.Mapping):
            return super().__new__(cls)
        elif isinstance(arg, abc.MutableSequence):
            return [cls(item) for item in arg]
        else:
            return arg
        
    def __init__(self, mapping):
        self.__data = {}
        for key, value in mapping.items():
            if iskeyword(key):
                key += '_'
            self.__data[key] = value

    def __getattr__(self, name):
        if hasattr(self.__data, name):
            return getattr(self.__data, name)
        else:
            return FrozenJSON(self.__data[name])

#     @classmethod
#     def build(cls, obj):
#         if isinstance(obj, abc.Mapping):
#             return cls(obj)
#         elif isinstance(objc, abc.MutableSequence):
#             return [cls.build(item) for item in obj]
#         else:
#             return obj
            

In [15]:
data = FrozenJSON(feed)
len(feed.Schedule.speakers)

357

In [10]:
sorted(data.Schedule.keys())

['conferences', 'events', 'speakers', 'venues']

In [11]:
for key, value in sorted(data.Schedule.items()):
    print(f'{len(value):3} {key}')

  1 conferences
484 events
357 speakers
 53 venues


In [33]:
import warnings

DB_NAME = 'data/schedule1_db'
CONFERENCE = 'conference.115'

class Record:
    def __init__(self, **kwargs):
        self.__dict__.update(kwargs)
        
def load_db(db):
#     raw_data = osconfeed.load()
    raw_data = feed
    warnings.warn('loading ' + DB_NAME)
    for collection, rec_list in raw_data['Schedule'].items():
        record_type = collection[:-1]
        cls_name = recort_type.capitalize()
        cls = globals().get(cls_name, DbRecord)
        if inspect.isclass(cls) and issublclass(cls, DbRecord):
            factory = cls
        else:
            factory = DbRecord
        for record in rec_list:
            key = '{}.{}'.format(record_type, record['serial'])
            record['serial'] = key
            db[key] = factory(**record)

In [35]:
import shelve

db = shelve.open(DB_NAME)
if CONFERENCE not in db:
    load_db(db)

In [32]:
class MissingDatabaseError(RuntimeError):
    """Raised when a database is required but was not set"""
    
class DbRecord(Record):
    __db = None
    
    @staticmethod
    def set_db(db):
        DbRecord.__db = db
        
    @staticmethod
    def get_db():
        return DbRecord.__db
    
    @classmethod
    def fetch(cls, ident):
        db = cls.get_db()
        try:
            return db[ident]
        except TypeError:
            if db is None:
                msg = "database not setup; call '{}.set_db(my_db)"
                raise MissingDatabaseError(msg.format(cls.__name__))
            else:
                raise
                
    def __repr__(self):
        if hasattr(self, 'serial'):
            cls_name = self.__class__.__name__
            return "{} serial={!r}>".format(cls_name, self.serial)
        else:
            return super().__repr__()
        
class Event(DbRecord):
    
    @property
    def venue(self):
        key = f'venue.{self.venue_serial}'
        return self.__class__.fetch(key)
    
    @property
    def speakers(self):
        if not hasattr(self, '_speaker_objs'):
            spkr_serials = self.__dict__['speakers']
            fetch = self.__class__.fetch
            self._speaker_objs = [fetch('speaker.{}'.format(key))
                                 for key in spkr_serials]
        return self._speaker_objs
    
    def __repr__(self):
        if hasattr(self, 'name'):
            cls_name = self.__class__.__name__
            return '<{} {!r}>'.format(cls_name, self.name)
        else:
            return super().__repr__()
        

In [36]:
class LineItem:
    
    def __init__(self, description, weight, price):
        self.description = description
        self.weight = weight
        self.price = price
        
    def subtotal(self):
        return self.weight * self.price
    
    @property
    def weight(self):
        return self.__weight
    
    @weight.setter
    def weight(self, value):
        if value > 0:
            self.__weight = value
        else:
            raise ValueError('value must be > 0')

In [2]:
# 'Quantity' property factory

def quantity(storage_name):
    def qty_getter(instance):
        return instance.__dict__[storage_name]
    
    def qty_setter(instance, value):
        if value > 0:
            instance.__dict__[storage_name] = value
        else:
            raise ValueError('value must be > 0')
            
    return property(qty_getter, qty_setter)

class LineItem:
    weight = quantity('weight')
    price = quantity('price')
    
    def __init__(self, description, weight, price):
        self.description = description
        self.weight = weight
        self.price = price
        
    def subtotal(self):
        return self.weight * self.price